# Building a web app

## 0. Minimal

In [1]:
from flask import Flask, request

app = Flask(__name__)

@app.route('/')
def root():
    """
    Simplest possible. Return a string.
    """
    return "Hello world!"

## 1. Getting an arg

In [ ]:
@app.route('/')
def root():
    """
    Get one argument as a string.
    """
    name = request.args.get('name', 'world')
    return f"Hello {name}!"

## 2. Calculator

In [ ]:
@app.route('/')
def root():
    """
    (2) A simple calculator with GET requests.
    """
    vp = float(request.args.get('vp') or 0)
    rho = float(request.args.get('rho') or 0)
    return f"Impedance: {vp * rho}"


## 3. ML classifier

To do this, we need some new functions:

- one to process the image as in training.
- one to fetch an image from a URL.
- one to make a prediction from an image.

First, here's the image processing function:

In [ ]:
def img_to_arr(img):
    """
    Apply the same processing we used in training: greyscale and resize.
    """
    img = img.convert(mode='L').resize((32, 32))
    return np.asarray(img).ravel() / 255

Here's the `fetch_image()` and `predict_from_image()` functions:

In [7]:
def fetch_image(url):
    """
    Download an image from the web and pass to the image processing function.
    """
    r = requests.get(url)
    f = BytesIO(r.content)
    return Image.open(f) 


def predict_from_image(clf, img):
    """
    Classify an image.
    """
    arr = img_to_arr(img)
    X = np.atleast_2d(arr)
    probs = clf.predict_proba(X)
    result = {
        'class': clf.classes_[np.argmax(probs)],
        'prob': probs.max(),
        'classes': clf.classes_.tolist(),
        'probs': np.squeeze(probs).tolist(), # Must be serializable.
    }
    return result

Now we can write the prediction function for the app:

In [6]:
import joblib
from flask import Flask, request, jsonify

app = Flask(__name__)

CLF = joblib.load('../app/data/rf.gz')

@app.route('/predict')
def predict():
    """
    (3) Make a prediction from a URL given via GET request.
    
    Using a URL means we can still just accept a string as an arg.

    There's still no human interface.
    """
    url = request.args.get('url')
    # result = utils.predict_from_url(CLF, url)
    
    # Deal with not getting a URL.
    if url:
        img = utils.fetch_image(url)
        result = utils.predict_from_image(CLF, img)
    else:
        result = 'Please provide a URL'

    return jsonify(result)

## 4. Get image via a form

First we'll just show a 'Hello world' page:

In [8]:
@app.route('/simple', methods=['GET'])
def simple():
    """
    (4a) Render a template.
    """
    return render_template('simple_page.html')

Now we can add a form:

In [ ]:
@app.route('/form', methods=['GET', 'POST'])
def form():
    """
    (4b) Make a prediction from a URL given via a form.
    """
    if request.method == 'POST':
        url = request.form.get('url')
        img = utils.fetch_image(url)
        result = utils.predict_from_image(CLF, img)
        result['url'] = url  # If we add this back, we can display it.
    else:
        result = {}

    return render_template('form.html', result=result)
